In [30]:
import pandas as pd
import jsonlines
from collections import Counter
import numpy as np
from tqdm import tqdm

In [29]:
! wc -l snli_1.0_train.jsonl
! wc -l snli_1.0_dev.jsonl
! wc -l snli_1.0_test.jsonl

550152 snli_1.0_train.jsonl
10000 snli_1.0_dev.jsonl
10000 snli_1.0_test.jsonl


In [31]:
def read_and_save_jsonl_file(file_path, where_to_save, total=None):
    data = pd.DataFrame([])
    label_mapper = {
        "entailment": 2,
        "neutral": 1,
        "contradiction": 0
    }
    first = True
    with jsonlines.open(file_path) as reader:
        for obj in tqdm(reader, total=total):
            score = obj['gold_label']
            if score == "-":
                labels_with_frequency = sorted(Counter(obj["annotator_labels"]).most_common(), key=lambda x: x[1], reverse=True)
                most_frequent_labels = list(
                    map(
                        label_mapper.__getitem__,
                        map(lambda x: x[0], filter(lambda x : x[1] == labels_with_frequency[0][1], labels_with_frequency))
                    )
                )
                score = np.median(most_frequent_labels)
            else:
                score = label_mapper[score]
            sent1 = obj['sentence1']
            sent2 = obj['sentence2']
            data = pd.DataFrame({"text": (sent1 + ' ' + sent2).replace('"', ''), "score": score}, index=[0])
            data.to_csv(where_to_save, mode='w' if first else 'a', header=first)
            if first:
                first = False
    return data

In [32]:
read_jsonl_file("snli_1.0_train.jsonl", total=550152).to_csv("train.csv", index=False)
read_jsonl_file("snli_1.0_dev.jsonl", total=10000).to_csv("val.csv", index=False)
read_jsonl_file("snli_1.0_test.jsonl", total=10000).to_csv("test.csv", index=False)

 61%|██████    | 336110/550152 [31:10<19:51, 179.68it/s]


KeyboardInterrupt: 